In [1]:
import pygam as pygam
import pandas as pd
import patsy as pt
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.tsa.arima import model
from pygam import LinearGAM, s, f, l

In [2]:
# Reading in and cleaning data

# Office of Field Operations data 2019 - 2023
OFOTwentyThree = pd.read_csv("https://raw.githubusercontent.com/jacobaschoff/midterm/main/nationwide-drugs-fy20-fy23%20(3).csv")
nineteen = pd.read_csv("https://raw.githubusercontent.com/jacobaschoff/midterm/main/nationwide-drugs-fy19-fy22.csv")
OFOTwnetyNineteen = nineteen[nineteen['FY']==2019]

# Air and Marine Operations data 2019 - 2023
#AMOTwentyThree = pd.read_csv("https://raw.githubusercontent.com/jacobaschoff/midterm/main/amo-drug-seizures-fy20-fy23.csv")
#AMOnineteen = pd.read_csv("https://raw.githubusercontent.com/jacobaschoff/midterm/main/amo-drug-seizures-fy19-fy22.csv")
#AMOTwentyNineteen = AMOnineteen[AMOnineteen['FY']==2019]

# Combining data and transferring to 

data = pd.concat([OFOTwnetyNineteen, OFOTwentyThree])

# Identifying cities for mapping the data

city_pattern = r'(.+)(?: FIELD OFFICE| SECTOR)'
data['City'] = data['Area of Responsibility'].str.extract(city_pattern)

# Matching cities with states

citiesandstates = pd.read_csv('https://raw.githubusercontent.com/jacobaschoff/midterm/main/uscities.csv')
citiesandstates = citiesandstates.rename(columns={'city_ascii' : 'City'})
citiesandstates = citiesandstates.rename(columns={'state_name' : 'State'})
citiesandstates['City'] = citiesandstates['City'].str.upper()

data = pd.merge(data, citiesandstates[['City', 'State']], on='City', how='left')
data

,FY,Month (abbv),Component,Region,Land Filter,Area of Responsibility,Drug Type,Count of Event,Sum Qty (lbs),City,State
0,2019,APR,Office of Field Operations,Coastal/Interior,Other,ATLANTA FIELD OFFICE,Ecstasy,3,8.379947,ATLANTA,Georgia
1,2019,APR,Office of Field Operations,Coastal/Interior,Other,ATLANTA FIELD OFFICE,Ecstasy,3,8.379947,ATLANTA,Texas
2,2019,APR,Office of Field Operations,Coastal/Interior,Other,ATLANTA FIELD OFFICE,Ecstasy,3,8.379947,ATLANTA,Illinois
3,2019,APR,Office of Field Operations,Coastal/Interior,Other,ATLANTA FIELD OFFICE,Ecstasy,3,8.379947,ATLANTA,Indiana
4,2019,APR,Office of Field Operations,Coastal/Interior,Other,ATLANTA FIELD OFFICE,Ecstasy,3,8.379947,ATLANTA,Michigan
...,...,...,...,...,...,...,...,...,...,...,...
42185,2023,SEP,U.S. Border Patrol,Southwest Border,Land Only,YUMA SECTOR,Methamphetamine,2,84.452600,YUMA,Colorado
42186,2023,SEP,U.S. Border Patrol,Southwest Border,Land Only,YUMA SECTOR,Methamphetamine,2,84.452600,YUMA,Tennessee
42187,2023,SEP,U.S. Border Patrol,Southwest Border,Land Only,YUMA SECTOR,Other Drugs**,2,1.748300,YUMA,Arizona
42188,2023,SEP,U.S. Border Patrol,Southwest Border,Land Only,YUMA SECTOR,Other Drugs**,2,1.748300,YUMA,Colorado


In [3]:
data = data.rename(columns={'Sum Qty (lbs)': 'Qty', 'Drug Type': 'Drug', 'Area of Responsibility': 'Office', 'Count of Event': 'Count'})
data.head()

,FY,Month (abbv),Component,Region,Land Filter,Office,Drug,Count,Qty,City,State
0,2019,APR,Office of Field Operations,Coastal/Interior,Other,ATLANTA FIELD OFFICE,Ecstasy,3,8.379947,ATLANTA,Georgia
1,2019,APR,Office of Field Operations,Coastal/Interior,Other,ATLANTA FIELD OFFICE,Ecstasy,3,8.379947,ATLANTA,Texas
2,2019,APR,Office of Field Operations,Coastal/Interior,Other,ATLANTA FIELD OFFICE,Ecstasy,3,8.379947,ATLANTA,Illinois
3,2019,APR,Office of Field Operations,Coastal/Interior,Other,ATLANTA FIELD OFFICE,Ecstasy,3,8.379947,ATLANTA,Indiana
4,2019,APR,Office of Field Operations,Coastal/Interior,Other,ATLANTA FIELD OFFICE,Ecstasy,3,8.379947,ATLANTA,Michigan


In [ ]:
data.dtypes

In [23]:
data['OfficeNum'] = pd.factorize(data['Office'])[0]
data['DrugNum'] = pd.factorize(data['Drug'])[0]
data.head(100)

,FY,Month (abbv),Component,Region,Land Filter,Office,Drug,Count,Qty,City,State,OfficeNum,DrugNum
0,2019,APR,Office of Field Operations,Coastal/Interior,Other,ATLANTA FIELD OFFICE,Ecstasy,3,8.379947,ATLANTA,Georgia,0,0
1,2019,APR,Office of Field Operations,Coastal/Interior,Other,ATLANTA FIELD OFFICE,Ecstasy,3,8.379947,ATLANTA,Texas,0,0
2,2019,APR,Office of Field Operations,Coastal/Interior,Other,ATLANTA FIELD OFFICE,Ecstasy,3,8.379947,ATLANTA,Illinois,0,0
3,2019,APR,Office of Field Operations,Coastal/Interior,Other,ATLANTA FIELD OFFICE,Ecstasy,3,8.379947,ATLANTA,Indiana,0,0
4,2019,APR,Office of Field Operations,Coastal/Interior,Other,ATLANTA FIELD OFFICE,Ecstasy,3,8.379947,ATLANTA,Michigan,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2019,APR,Office of Field Operations,Coastal/Interior,Other,LOS ANGELES FIELD OFFICE,Cocaine,24,90.790703,LOS ANGELES,California,4,4
96,2019,APR,Office of Field Operations,Coastal/Interior,Other,LOS ANGELES FIELD OFFICE,Cocaine,24,90.790703,LOS ANGELES,Texas,4,4
97,2019,APR,Office of Field Operations,Coastal/Interior,Other,LOS ANGELES FIELD OFFICE,Ecstasy,20,2.787084,LOS ANGELES,California,4,0
98,2019,APR,Office of Field Operations,Coastal/Interior,Other,LOS ANGELES FIELD OFFICE,Ecstasy,20,2.787084,LOS ANGELES,Texas,4,0


In [24]:
data.dtypes

FY                int64
Month (abbv)     object
Component        object
Region           object
Land Filter      object
Office           object
Drug             object
Count             int64
Qty             float64
City             object
State            object
OfficeNum         int64
DrugNum           int64
dtype: object

In [38]:
data.Drug.unique()
#data.DrugNum.unique()

array(['ATLANTA FIELD OFFICE', 'BALTIMORE FIELD OFFICE',
       'CHICAGO FIELD OFFICE', 'HOUSTON FIELD OFFICE',
       'LOS ANGELES FIELD OFFICE', 'MIAMI FIELD OFFICE',
       'NEW ORLEANS FIELD OFFICE', 'NEW YORK FIELD OFFICE',
       'PRECLEARANCE FIELD OFFICE', 'SAN FRANCISCO FIELD OFFICE',
       'SAN JUAN FIELD OFFICE', 'TAMPA FIELD OFFICE',
       'BOSTON FIELD OFFICE', 'BUFFALO FIELD OFFICE',
       'DETROIT FIELD OFFICE', 'SEATTLE FIELD OFFICE',
       'PORTLAND FIELD OFFICE', 'EL PASO FIELD OFFICE',
       'LAREDO FIELD OFFICE', 'SAN DIEGO FIELD OFFICE',
       'TUCSON FIELD OFFICE', 'MIAMI SECTOR', 'BLAINE SECTOR',
       'BUFFALO SECTOR', 'DETROIT SECTOR', 'HAVRE SECTOR',
       'HOULTON SECTOR', 'SPOKANE SECTOR', 'SWANTON SECTOR',
       'BIG BEND SECTOR', 'DEL RIO SECTOR', 'EL CENTRO SECTOR',
       'EL PASO SECTOR', 'LAREDO SECTOR', 'RIO GRANDE VALLEY SECTOR',
       'SAN DIEGO SECTOR', 'TUCSON SECTOR', 'YUMA SECTOR',
       'NEW ORLEANS SECTOR', 'RAMEY SECTOR', 'GRAND FO

In [25]:
regression = smf.ols("OfficeNum ~ -1  + FY +  Count + Qty + DrugNum", data=data)

In [26]:
xgam = data[['FY','Count','Qty','DrugNum']]
ygam = data[['OfficeNum']]

In [27]:
model = """OfficeNum ~ -1  + FY +  Count + Qty + DrugNum"""
y,x = pt.dmatrices(model, data=data)

In [41]:
model = LinearGAM(s(0) + s(1) + s(2) + s(3))
modelFit = model.gridsearch(np.asarray(x), y)

100% (11 of 11) |########################| Elapsed Time: 0:00:12 Time:  0:00:120:01


In [42]:
# Import plotly tools to create a grid of subplots (figures) that work together
from plotly import tools
import plotly.offline as py
import plotly.graph_objs as go

# Name each figure
titles = ['year', 'Count', 'Qty', 'Drug']

# Create the subplots in a single-row grid
fig = tools.make_subplots(rows=1, cols=4, subplot_titles=titles)
# Dictate the size of the figure, title, etc.
fig['layout'].update(height=500, width=1000, title='pyGAM', showlegend=False)

# Loop over the titles, and create the corresponding figures
for i, title in enumerate(titles):
    # Create the grid over which to estimate the effect of parameters
    XX = modelFit.generate_X_grid(term=i)
    # Calculate the value and 95% confidence intervals for each parameter
    # This will become the expected effect on the dependent variable for a given value of x
    pdep, confi = modelFit.partial_dependence(term=i, width=.95)
    
    # Create the effect and confidence interval traces (there are 3 total)
    trace = go.Scatter(x=XX[:,i], y=pdep, mode='lines', name='Effect')
    ci1 = go.Scatter(x = XX[:,i], y=confi[:,0], line=dict(dash='dash', color='grey'), name='95% CI')
    ci2 = go.Scatter(x = XX[:,i], y=confi[:,1], line=dict(dash='dash', color='grey'), name='95% CI')

    # Add each of the three traces to the figure in the relevant grid position
    fig.append_trace(trace, 1, i+1)
    fig.append_trace(ci1, 1, i+1)
    fig.append_trace(ci2, 1, i+1)

#Plot the figure
py.iplot(fig)


C:\Users\ryanh\anaconda3\Lib\site-packages\plotly\tools.py:460: DeprecationWarning:

plotly.tools.make_subplots is deprecated, please use plotly.subplots.make_subplots instead

